In [67]:
import datasets
from datasets import load_dataset

dataset = load_dataset('deepmind/narrativeqa')

dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 32747
    })
    test: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 10557
    })
    validation: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 3461
    })
})

In [68]:
from IPython.display import display, Markdown

df = dataset['validation'].to_pandas()

display(df.sample(1).iloc[0].question)

{'text': 'Steve Miller is a camerarman for what company?',
 'tokens': array(['Steve', 'Miller', 'is', 'a', 'camerarman', 'for', 'what',
        'company', '?'], dtype=object)}

In [69]:
output = df.sample(1).iloc[0].document["text"]
with open("output.txt", "w") as file:
	file.write(output)

In [70]:
df['context'] = df['document'].apply(lambda x: x['text'])
df['question'] = df['question'].apply(lambda x: x['text'])
df = df.rename(columns={'question': 'questions'})
df = df.groupby('context').agg({'questions': list}).reset_index()

In [71]:
# from IPython import display
# # Filter out html documents - we are only interested in text documents
# df = df.assign(html=df['context'].str.contains('<html>'))
# df['html'].value_counts()
# df = df[~df['html']].reset_index(drop=True)

In [72]:
df['context_length'] = df['context'].apply(len)

In [73]:
count_greater_than_million = (df['context_length'] > 1000000).sum()
count_greater_than_million

6

In [74]:
# Get first 50 rows
df = df.iloc[:50]
df.shape

(50, 3)

In [75]:
df.describe()

,context_length
count,50.000000
mean,217480.920000
std,58733.594823
min,110830.000000
25%,186267.000000
50%,215859.500000
75%,240608.500000
max,417011.000000


In [76]:
from docling.document_converter import DocumentConverter, DocumentStream
from io import BytesIO

def convert_html_to_text(html):
	converter = DocumentConverter()

	# Convert document
	buf = BytesIO(html.encode('utf-8'))
	source = DocumentStream(name="my_doc.html", stream=buf)
	doc = converter.convert(source)
	html = doc.document.export_to_text()
	return html

df['context_text'] = df['context'].apply(convert_html_to_text)
display(df['context_text'].iloc[0])
df['context_text_length'] = df['context_text'].apply(len)

detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested tables: skipping for now
detected nested 

'```\nD      O       M       I       N       O\n\n                                       By\n                                          Richard Kelly\n\n                                                     September 8, 2004\n\n     [NOTE: THE ORIGINAL TEXT OF THIS SCRIPT HAD NUMBERED SCENES\n     AND SOME "OMITTED" SCENE SLUGLINES, WHICH HAVE NOT BEEN\n     RETAINED FOR THE ONLINE TEXT.]\n\n     OVER STUDIO LOGO:\n\n     VOICES. VOICES CHATTERING in some sort of public place.\n\n                          ROBOTIC FEMALE VOICE (V.O.)\n                Now serving G233. Please report\n                to window four.\n\n     VOICES are broken by someone coughing up phlegm.\n\n     TITLE CARDS:   THIS FILM IS BASED ON A TRUE STORY (BEAT)\n\n                                 (SORT OF)\n\n     INT. DMV -- AFTERNOON\n\n     The DEPARTMENT OF MOTOR VEHICLES. SOUTH CENTRAL LOS ANGELES.\n     People are jam-packed into the internal maze of cubicles. It\n     doesn\'t get more crowded and disgusting 

In [77]:
df.describe()

,context_length,context_text_length
count,50.000000,50.000000
mean,217480.920000,4494.800000
std,58733.594823,31570.903566
min,110830.000000,30.000000
25%,186267.000000,30.000000
50%,215859.500000,30.000000
75%,240608.500000,30.000000
max,417011.000000,223270.000000


In [56]:
output = df.sample(1).iloc[0]
with open("output.txt", "w") as file:
	file.write(output.context)
# with open("output.txt", "w") as file:
# 	file.write(output.context_text)

In [24]:
missing_text_count = df['context_text'].apply(lambda x: 1 if '<missing-text>' in x else 0).sum()
missing_text_count

49

In [23]:
html_count = df['context'].str.startswith('<html>').sum()
html_count

50